#### Defined by User

In [ ]:
## specify your Dkube username
DKUBEUSERNAME = "ocdkube"

## Define the model monitor name here that you will be creating 
MONITOR_NAME = "insurance-mm-kf"

DATA_SOURCE = "local"

# Required in train.ipynb to retrain the model
DKUBE_TRAINING_CODE_NAME = "insurance"

LIVE_DATASET =  MONITOR_NAME+'-s3'

## Dkube information
import os
TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")
DKUBE_URL = os.getenv("DKUBE_URL","")

## AWS_S3 
S3_KEY = ''
S3_SECRET_KEY = ''
S3_ENDPOINT = ''
RETRAINING_DATASET = 'insurance-training-data'
## Inference URL
INFERENCE_URL = None

# dataset to be used as training data
DKUBE_BASE_DATASET = "insurance-data"

# Model name to be created or used for example, it will create the model
# if not existing otherwise it will ignore creation.
MODEL_NAME = MONITOR_NAME

# the frequency with which monitoring will run, value will be considered in minutes
RUN_FREQUENCY = 5

if not(TOKEN or DKUBEUSERNAME or DKUBE_URL):
    print("Please fill the Dkube details first (TOKEN, DKUBE_URL, DKUBEUSERNAME)")
    raise TypeError
if not (S3_KEY or S3_SECRET_KEY or S3_ENDPOINT):
    print("Please fill the AWS_S3 details first (ACCESS_KEY, SECRET_KEY)")
    raise TypeError

#### Dkube Resources

In [ ]:
import time,json
from dkube.sdk import *

In [ ]:
api = DkubeApi(URL=DKUBE_URL,token=TOKEN)
if DKUBEUSERNAME == api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

In [ ]:
data = api.get_modelmonitor_id(MONITOR_NAME)
if data.data:
    MONITOR_ID = data.data.get(MONITOR_NAME)
    if MONITOR_ID:
        raise ValueError(f"{MONITOR_NAME} already existing please use a different name")

In [ ]:
INPUT_TRAIN_TYPE = "training"
d3_config = {"MONITOR_NAME":MONITOR_NAME, "DATA_SOURCE": DATA_SOURCE, 
            "INPUT_TRAIN_TYPE":INPUT_TRAIN_TYPE, "DKUBEUSERNAME":DKUBEUSERNAME,
            "TOKEN":TOKEN, "DKUBE_URL":DKUBE_URL, "S3_KEY":S3_KEY,
            "S3_SECRET_KEY":S3_SECRET_KEY, "S3_ENDPOINT":S3_ENDPOINT,
            "DKUBE_BASE_DATASET":DKUBE_BASE_DATASET, "MODEL_NAME":MODEL_NAME,
            "RETRAINING_DATASET":RETRAINING_DATASET, "RUN_FREQUENCY":RUN_FREQUENCY,
            "DKUBE_TRAINING_CODE_NAME":DKUBE_TRAINING_CODE_NAME,
            "LIVE_DATASET":LIVE_DATASET}
%store d3_config

#### Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=DKUBE_BASE_DATASET)
    dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
    dataset.update_dataset_source(source="pub_url")
    api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")
        
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=RETRAINING_DATASET)
    dataset.update_dataset_source(source="dvs")
    api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

### Model Monitor Datasets

##### Predict and labelled Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=LIVE_DATASET,remote=True)
    dataset.update_dataset_source('s3')
    dataset.update_s3_details(
        endpoint = S3_ENDPOINT,
        bucket="modelmonitor",
        prefix='',
        key=S3_KEY,
        secret=S3_SECRET_KEY)
    api.create_dataset(dataset)

except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e


#### Model

In [ ]:
try:
    model = DkubeModel(DKUBEUSERNAME, name=MODEL_NAME)
    model.update_model_source(source='dvs')
    api.create_model(model)
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    api.delete_dataset(DKUBEUSERNAME,DKUBE_BASE_DATASET,force=True)
    api.delete_dataset(DKUBEUSERNAME,S3_DATASET,force=True)
    api.delete_model(DKUBEUSERNAME,MODEL_NAME,force=True)
    %store -d d3_config